In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Personality and Habits of The Customer Analysis

This is my learning notebook about personality and habit of the customer. The main goal of this project is to know personality of the customer and their spending habit. Also, it is help me to gain more understanding about exploratory data analysis and clustering. All code in this project was inspired by kylethomson2, shout out to him and you can check his work on the link below.

The project itself start from generate business question to guide us. Then data preparation, we did some feature engineering and data cleaning. After that we start make cluster from our data to know our customer personality. After we know about our customer personality we can analyze their habits.

source : https://www.kaggle.com/kylethomson2/customer-personality-eda

## Business Question

1. Who is our customer?
2. How is their spending Habit?

## Library Import

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date

import warnings
warnings.filterwarnings('ignore')

In [ ]:
path ='../input/customer-personality-analysis/marketing_campaign.csv'
dataLoad = pd.read_csv(path, sep='\t', index_col='ID')
dataLoad.head()

In [ ]:
dataLoad.info()

In [ ]:
dataLoad.isnull().sum()

In [ ]:
data = dataLoad.copy()
dataCust = data.dropna()
dataCust.isnull().sum()

In [ ]:
def get_age(birthyear):
    return date.today().year-birthyear

dataCust['Age'] = dataCust.Year_Birth.map(get_age)
dataCust['Age'].describe()  

In [ ]:
dataCust.sort_values(by='Age', ascending=False).head()

In [ ]:
dataCust.drop([11004, 1150, 7829], inplace=True)
dataCust['Age'].describe()

In [ ]:
dataCust['Spending'] =dataCust.MntWines+dataCust.MntFruits+dataCust.MntMeatProducts+dataCust.MntFishProducts+dataCust.MntSweetProducts+dataCust.MntGoldProds
dataCust.head()

In [ ]:
dataCust['Time_with_company'] = pd.to_datetime(dataCust.Dt_Customer, dayfirst=True, format='%d-%m-%Y')
dataCust.head()

In [ ]:
dataCust['Time_with_company'] = pd.to_numeric(dataCust.Time_with_company.dt.date.apply(lambda z : (date.today()-z)).dt.days, 
downcast= 'integer')/30
dataCust.head()

In [ ]:
dataCust.Education.unique()

In [ ]:
dataCust.Marital_Status.unique()

In [ ]:
dataCust.Marital_Status = dataCust.Marital_Status.replace({'Together':'Partner', 'Married':'Partner', 'Divorced':'Single', 'Widow':'Single', 'Alone':'Single', 'Absurd':'Single', 'YOLO':'Single'})
dataCust.head()

In [ ]:
dataCust['Children']=dataCust.Kidhome + dataCust.Teenhome
dataCust['Has_Children'] = np.where(dataCust.Children > 0, 'Has Child', 'No child') 
dataCust.head()

In [ ]:
dataCust = dataCust.rename(columns={'MntWines':'Wines', 'MntFruits':'Fruits', 'MntMeatProducts':'Meat','MntFishProducts':'Fish', 'MntSweetProducts':'Sweets', 'MntGoldProds':'Gold'})
dataCust.head()

In [ ]:
dataCust = dataCust.rename(columns={'NumWebPurchases':'Web', 
                                   'NumCatalogPurchases': 'Catalogue',
                                   'NumStorePurchases': 'Store', 
                                   'NumWebVisitsMonth' : 'WebVisit'})
dataCust.head()

In [ ]:
dataCust.Web.value_counts()

In [ ]:
outlierID = dataCust.loc[dataCust.Web > 20].index
dataCust.drop(outlierID, inplace=True)
dataCust.Web.value_counts()

In [ ]:
dataCust.Catalogue.describe()

In [ ]:
dataCust.Catalogue.value_counts()

In [ ]:
outlierId = dataCust.loc[dataCust.Catalogue > 20].index
dataCust.drop(outlierId, inplace=True)


In [ ]:
dataCust.Catalogue.describe()

In [ ]:
dataCust.Store.describe()

In [ ]:
dataCust.Store.value_counts()

In [ ]:
demographic_data = dataCust[['Education', 'Marital_Status', 
                             'Has_Children', 'Children', 
                             'Age', 'Income', 'Spending',
                            'Time_with_company', 'Wines', 'Fruits', 'Sweets', 'Meat',
                            'Web', 'Catalogue', 'Store', 'WebVisit']]
demographic_data.head()

## Analysis

In [ ]:
ax = sns.countplot(data=demographic_data, x='Education', palette=('Set2'))
ax.set(xlabel=None, title='Education Level of Customer')

In [ ]:
ax = sns.countplot(data= demographic_data, x='Marital_Status', palette='husl')
ax.set(xlabel=None, title='Marital Status of Customer')

In [ ]:
ax = sns.countplot(data=demographic_data, x='Has_Children', palette='flare')
ax.set(xlabel=None, title='Child Status of Customer')

In [ ]:
ax = sns.histplot(data=demographic_data.Age)
ax.set(xlabel='Age', ylabel=None, title = 'Age of Customer')
ax.grid(axis='x')

In [ ]:
demographic_data.Age.describe()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(25, 8))
sns.histplot(ax=ax[0], data=demographic_data.Income, color='midnightblue')
sns.histplot(ax=ax[1], data=demographic_data.Spending, color='midnightblue')

ax[0].set_title('Income of Customer')
ax[0].set_xlabel('Income  (USD)')

ax[1].set_title('Spending of Customer')
ax[1].set_xlabel('Spending (USD)')

In [ ]:
demographic_data.Income.describe()

In [ ]:
demographic_data.Spending.describe()

In [ ]:
ax = sns.histplot(data=demographic_data.Time_with_company, color='darkslateblue')
ax.set(title='Time with company',
      xlabel = 'Months',
      ylabel = None)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(25, 15))

sns.histplot(ax=ax[0, 0], data=demographic_data.Wines, color='darkorange')
sns.histplot(ax=ax[0, 1], data=demographic_data.Meat, color='seagreen')
sns.histplot(ax=ax[1, 0], data=demographic_data.Fruits, color='lightcoral')
sns.histplot(ax=ax[1, 1], data=demographic_data.Sweets, color='mediumturquoise')

fig.suptitle('Items Bought by Customer')

In [ ]:
demographic_data.Wines.describe()

In [ ]:
demographic_data.Meat.describe()

In [ ]:
demographic_data.Fruits.describe()

In [ ]:
demographic_data.Sweets.describe()

## Creating Cluster

In this section I will cluster customer based of their:
1. Income
2. Time with company
3. Spending
    
The four cluster itself will consist of:

1. **Star** --> Customer that have High Income, High Spending, and long time with company.
2. **High Potential** --> Customer that have High Income, High Spending, and short time with company.
3. **Needs Attention** --> Customer that have Low Income, Low Spending, and short time with company
4. **Leaky Bucket** --> Customer that have Low Income, Low Spending and long time with company.

In [ ]:
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.mixture import GaussianMixture

In [ ]:
scaler = StandardScaler()
temp = demographic_data[['Income', 'Spending', 'Time_with_company']]

X_std = scaler.fit_transform(temp)
X = normalize(X_std, norm='l2')

In [ ]:
gmm = GaussianMixture(n_components=4, covariance_type='spherical', random_state=0, max_iter=1000).fit(X)
labels = gmm.predict(X)
temp['Cluster'] = labels
temp.head()

In [ ]:
temp = temp.replace({0: 'Leaky Buckets',
                     1:'High Potential',
                     2:'Need Attention',
                     3:'Stars'})
temp.head()

In [ ]:
demographic_data = demographic_data.merge(temp.Cluster, left_index=True, right_index=True)

In [ ]:
pd.options.display.float_format='{:.0f}'.format
summary = demographic_data[['Income', 'Time_with_company', 'Spending', 'Cluster']]

summary.set_index('Cluster', inplace=True)
summary = summary.groupby('Cluster').describe().transpose()
summary

In [ ]:
ax = sns.countplot(data=demographic_data, y='Cluster', palette='muted')
ax.bar_label(container=ax.containers[0], padding=-35)
ax.set_title('Cluster Distribution')

In [ ]:
ax = sns.scatterplot(x=demographic_data.Income, y=demographic_data.Spending, hue= demographic_data.Cluster, palette='muted')

In [ ]:
ax = sns.scatterplot(x=demographic_data.Time_with_company,
                y=demographic_data.Spending, hue=demographic_data.Cluster,
                palette='muted')

In [ ]:
ax = sns.scatterplot(x=demographic_data.Time_with_company,
                y=demographic_data.Income, hue=demographic_data.Cluster,
                palette='muted')

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20, 10))

sns.countplot(ax=ax[0, 0], data=demographic_data.loc[demographic_data.Cluster == 'Stars'], y='Education',
             order=['PhD', "Master", "Graduation", "2n Cycle"], palette='Set2')

sns.countplot(ax=ax[0, 1], data=demographic_data.loc[demographic_data.Cluster == 'High Potential'],
             y='Education', order=['PhD', "Master", "Graduation", "2n Cycle"], palette='Set2')

sns.countplot(ax=ax[1, 0], data=demographic_data.loc[demographic_data.Cluster == 'Need Attention'],
             y='Education', order=['PhD', "Master", "Graduation", "2n Cycle"], palette='Set2')

sns.countplot(ax=ax[1, 1], data=demographic_data.loc[demographic_data.Cluster == 'Leaky Buckets'],
             y='Education', order=['PhD', "Master", "Graduation", "2n Cycle"], palette='Set2')

ax[0, 0].set_title('Stars')
ax[0, 1].set_title('High Potential')
ax[1, 0].set_title('Need Attention')
ax[1, 1].set_title('Leaky Buckets')

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(25, 20))

stars = demographic_data.loc[demographic_data.Cluster == 'Stars']
highPoten = demographic_data.loc[demographic_data.Cluster == 'High Potential']
needAtten = demographic_data.loc[demographic_data.Cluster == 'Need Attention']
leakBuck = demographic_data.loc[demographic_data.Cluster == 'Leaky Buckets']

sns.histplot(ax=ax[0, 0], data=stars.Web, label='Web purchase', color='skyblue')
sns.histplot(ax=ax[0, 0], data=stars.Catalogue, label='Catalogue purchase', color='red')
sns.histplot(ax=ax[0, 0], data=stars.Store, label='Store purchase', color='gold')

sns.histplot(ax=ax[0, 1], data=highPoten.Web, label='Web purchase', color='skyblue')
sns.histplot(ax=ax[0, 1], data=highPoten.Catalogue, label='Catalogue purchase', color='red')
sns.histplot(ax=ax[0, 1], data=highPoten.Store, label='Store purchase', color='gold')

sns.histplot(ax=ax[1, 0], data=needAtten.Web, label='Web purchase', color='skyblue')
sns.histplot(ax=ax[1, 0], data=needAtten.Catalogue, label='Catalogue purchase', color='red')
sns.histplot(ax=ax[1, 0], data=needAtten.Store, label='Store purchase', color='gold')

sns.histplot(ax=ax[1, 1], data=leakBuck.Web, label='Web purchase', color='skyblue')
sns.histplot(ax=ax[1, 1], data=leakBuck.Catalogue, label='Catalogue purchase', color='red')
sns.histplot(ax=ax[1, 1], data=leakBuck.Store, label='Store purchase', color='gold')


fig.suptitle('Customer Habit by Its Cluster')

ax[0, 0].set_title('Stars')
ax[0, 1].set_title('High Potential')
ax[1, 0].set_title('Need Attention')
ax[1, 1].set_title('Leaky Bucket')

ax[0, 0].legend()
ax[0, 1].legend()
ax[1, 0].legend()
ax[1, 1].legend()

In [ ]:
ax = sns.barplot(x=demographic_data.Cluster, y = demographic_data.WebVisit)


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18, 9))

sns.swarmplot(ax=ax[0], y=demographic_data.Spending, x=demographic_data.Has_Children)
sns.swarmplot(ax=ax[1], y=demographic_data.Spending, x=demographic_data.Marital_Status)

In [ ]:
temp = demographic_data.loc[:, ['Wines', 'Meat', 'Fruits', 'Sweets', 'Cluster']]
temp = temp.groupby(by='Cluster').sum()
temp

In [ ]:
ax=temp.transpose().plot(kind='barh', stacked=True, colormap='Set2')


In [ ]:
ax = sns.scatterplot(y=demographic_data.Wines, x=demographic_data.Income, hue=demographic_data.Has_Children)

In [ ]:
demographic_data.loc[demographic_data.Has_Children == 'No child'].Wines.describe()

In [ ]:
demographic_data.loc[demographic_data.Has_Children == 'Has Child'].Wines.describe()

In [ ]:
demographic_data.loc[demographic_data.Has_Children == 'No child'].Wines.sum()

In [ ]:
demographic_data.loc[demographic_data.Has_Children == 'Has Child'].Wines.sum()